In [2]:
pwd

'C:\\Users\\Kosmo_24'

In [54]:
# 케라스 라이브러리 임포트

from keras.datasets import reuters  # 로이터 뉴스기사 데이터 임포트
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from keras.preprocessing import sequence # 기사별 단어 숫자 통일
from keras.utils import np_utils

In [53]:
# 기타 라이브러리 임포트

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


In [18]:
# seed값 설정
seed=0
np.random.seed(seed)
tf.random.set_seed(seed)

# 1. 데이터 확인

In [14]:
# 로이터 뉴스 데이터 가져온 후, 단어 기준으로 자르기(toknize)하고
# 불러 온 데이터를 학습셋, 테스트셋으로 나누기

(X_train, Y_train), (X_test, Y_test) = reuters.load_data(num_words=1000, test_split=0.2)

In [16]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))


8982
8982
2246
2246


In [20]:
import nltk # natural language tool kit
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kosmo_24\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:
# 형태소 분석기 사용

from nltk.tokenize import word_tokenize
word_tokenize("안녕하세요 저의 이름은 조성은입니다.")

['안녕하세요', '저의', '이름은', '조성은입니다', '.']

In [23]:
# konlpy 사용하면 한글 형태소 분석, mecab이 최근 핫함

In [27]:
# 카테고리 개수 세기

print("총", len(Y_train), "개") # 중복 포함 카테고리 수 
category = np.max(Y_train) + 1
print(category, '카테고리')

총 8982 개
46 카테고리


In [29]:
# 데이터 확인하기

print(len(X_train), "학습용 뉴스 기사")
print(len(X_test), "학습용 뉴스 기사")

8982 학습용 뉴스 기사
2246 학습용 뉴스 기사


In [32]:
# 첫번째 뉴스 기사 살펴보기

print(len(X_train[0])) # 단어 개수가 87개가 들어있는 뉴스기사 
X_train[0]

# 로이터 데이터는 tokenize()함수를 이용해서 뉴스기사를 형태소 분석한 후에 단어 빈도수 

# 딥러닝은 단어 그대로학습 x, 숫자로 변환한 다음에 학습해야 한다.

# 여기서는 데이터 안에서 해당 단어의 빈도수 순위를 붙인 것. 1이면 첫번째 빈도수 

87


[1,
 2,
 2,
 8,
 43,
 10,
 447,
 5,
 25,
 207,
 270,
 5,
 2,
 111,
 16,
 369,
 186,
 90,
 67,
 7,
 89,
 5,
 19,
 102,
 6,
 19,
 124,
 15,
 90,
 67,
 84,
 22,
 482,
 26,
 7,
 48,
 4,
 49,
 8,
 864,
 39,
 209,
 154,
 6,
 151,
 6,
 83,
 11,
 15,
 22,
 155,
 11,
 15,
 7,
 48,
 9,
 2,
 2,
 504,
 6,
 258,
 6,
 272,
 11,
 15,
 22,
 134,
 44,
 11,
 15,
 16,
 8,
 197,
 2,
 90,
 67,
 52,
 29,
 209,
 30,
 32,
 132,
 6,
 109,
 15,
 17,
 12]

In [35]:
print(len(Y_train))
Y_train[10:46]

8982


array([ 3,  3,  4,  4, 19,  8, 16,  3,  3, 21, 11,  4,  4,  3,  3,  1,  3,
        1,  3, 16,  1,  4, 13, 20,  1,  4,  4, 11,  3,  3,  3, 11, 16,  4,
        4, 20], dtype=int64)

첫 번째 기사 -> 87개 단어, 다른 것들은 150단어가 되면 형평성 x

단어 수 맞추기 100개로 맞추기 

sequence 라이브러리(클래스)의 pad_sequence 메소드(함수)를 사용함
(라이브러리 = 패키지 = 클래스 = 모듈) 가져다쓰는 파이썬 코드 (메소드 = 함수)

100개 단어 이상인 뉴스기사는 100개로 자르고, 100개 이상은 버린다.

100개 단어 이하인 뉴스기사는(87개) 13개는 0으로 채운다. -> max_len=100


In [38]:
x_train = sequence.pad_sequences(X_train, maxlen=100)
x_test = sequence.pad_sequences(X_test, maxlen=100)


In [40]:
x_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         1,   2,   2,   8,  43,  10, 447,   5,  25, 207, 270,   5,   2,
       111,  16, 369, 186,  90,  67,   7,  89,   5,  19, 102,   6,  19,
       124,  15,  90,  67,  84,  22, 482,  26,   7,  48,   4,  49,   8,
       864,  39, 209, 154,   6, 151,   6,  83,  11,  15,  22, 155,  11,
        15,   7,  48,   9,   2,   2, 504,   6, 258,   6, 272,  11,  15,
        22, 134,  44,  11,  15,  16,   8, 197,   2,  90,  67,  52,  29,
       209,  30,  32, 132,   6, 109,  15,  17,  12])

# y값 전처리 : 원핫인코딩

In [57]:
# Y데이터에 원핫 인코딩
y_train=np_utils.to_categorical(Y_train)
y_test=np_utils.to_categorical(Y_test)

# 딥러닝 구조 짜기(LSTM)

# LSTM 기본 뼈대 짜기

model = Sequential()
model.add(Embedding(불러온 단어 총 개수, 기사당 단어 수))
model.add(기사 당 단어 수, 기타 옵션) # LSTM이 RNN의 gradient 문제 해결
model.add(Dense())

Embedding : 총 1000개의 단어 중 100개의 단어가 들어옴
LSTM : 100개 노드(기사당 단어 수), activation: tanh
Dense: 출력 46개 노드(카테고리 수), activation: softmax

In [50]:
# 모델의 설정
model=Sequential()
model.add(Embedding(1000, 100)) # 기사 단어 수는 100개
model.add(LSTM(100, activation='tanh'))
model.add(Dense(46, activation='softmax'))

In [51]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [58]:
history = model.fit(x_train, y_train, batch_size=100, epochs=20, validation_data=(x_test, y_test))

C:\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 8982 samples, validate on 2246 samples
Epoch 1/20
8982/8982 [==============================] - 14s 2ms/step - loss: 2.5574 - accuracy: 0.3670 - val_loss: 2.0936 - val_accuracy: 0.4938
Epoch 2/20
8982/8982 [==============================] - 13s 1ms/step - loss: 2.0053 - accuracy: 0.4890 - val_loss: 1.9193 - val_accuracy: 0.5098
Epoch 3/20
8982/8982 [==============================] - 13s 1ms/step - loss: 1.9804 - accuracy: 0.5035 - val_loss: 1.9805 - val_accuracy: 0.5116
Epoch 4/20
8982/8982 [==============================] - 13s 1ms/step - loss: 1.7963 - accuracy: 0.5450 - val_loss: 1.7401 - val_accuracy: 0.5695
Epoch 5/20
8982/8982 [==============================] - 13s 1ms/step - loss: 1.7064 - accuracy: 0.5639 - val_loss: 1.8843 - val_accuracy: 0.5557
Epoch 6/20
8982/8982 [==============================] - 13s 1ms/step - loss: 1.6705 - accuracy: 0.5759 - val_loss: 1.6837 - val_accuracy: 0.5793
Epoch 7/20
8982/8982 [==============================] - 13s 1ms/step - loss: 1.585